You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [15]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [16]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [17]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [18]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [19]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [20]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [21]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources. The text should be in Spanish language.",
    agent=planner,
)

### Task: Write

In [22]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs. The text should be in Spanish language.",
    agent=writer,
)

### Task: Edit

In [23]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs. The text should be in Spanish language.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [24]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-12-10 04:24:29,286 - 140189941119872 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [25]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Ahora puedo dar una respuesta excelente

Final Answer: 

Plan de Contenido sobre Inteligencia Artificial

Título: Descubriendo el Futuro: Tendencias, Jugadores Clave y Noticias Relevantes sobre la Inteligencia Artificial

Introducción:
En la era digital actual, la Inteligencia Artificial (IA) ha revolucionado la forma en que interactuamos con la tecnología. Desde asistentes virtuales hasta sistemas de recomendación, la IA está presente en nuestras vidas de diversas formas. En este artículo, exploraremos las últimas tendencias, los principales actores del

I now can give a great answer

Final Answer:
# Descubriendo el Futuro: Tendencias, Jugadores Clave y Noticias Relevantes sobre la Inteligencia Artificial

En la era actual de la transformación digital, la Inteligencia Artificial (IA) se ha convertido en un elemento fundamental en nuestra interacción con la tecnología. Desde asistentes virtuales hasta sistemas de recomendación, la IA ha permeado todos los aspectos de nuestra vida diaria. En este artículo, exploraremos las últimas tendencias, los actores clave en el mercado y las noticias más relevantes en el campo de la IA.

## Últimas tendencias en Inteligencia Artificial

Los avances en el aprendizaje profundo y el procesamiento del lenguaje natural han llevado a un aumento significativo en la eficacia de los sistemas de IA. Estos avances se reflejan en la diversidad de aplicaciones de IA en sectores como la salud, el comercio electrónico y la automoción. Además, el desarrollo de IA ética y responsable se ha convertido en una priorida

- Display the results of your execution as markdown in the notebook.

In [26]:
from IPython.display import Markdown
Markdown(result)

# Descubriendo el Futuro: Tendencias, Jugadores Clave y Noticias Relevantes sobre la Inteligencia Artificial

En la era actual de la transformación digital, la Inteligencia Artificial (IA) se ha convertido en un elemento fundamental en nuestra interacción con la tecnología. Desde asistentes virtuales hasta sistemas de recomendación, la IA ha permeado todos los aspectos de nuestra vida diaria. En este artículo, exploraremos las últimas tendencias, los actores clave en el mercado y las noticias más relevantes en el campo de la IA.

## Últimas tendencias en Inteligencia Artificial

Los avances en el aprendizaje profundo y el procesamiento del lenguaje natural han llevado a un aumento significativo en la eficacia de los sistemas de IA. Estos avances se reflejan en la diversidad de aplicaciones de IA en sectores como la salud, el comercio electrónico y la automoción. Además, el desarrollo de IA ética y responsable se ha convertido en una prioridad para garantizar que el impacto de esta tecnología sea beneficioso para la sociedad en su conjunto.

## Jugadores clave en el mercado de la Inteligencia Artificial

Empresas líderes en IA como Google, Amazon y Microsoft han establecido su posición dominante en el mercado. Sin embargo, las startups emergentes también están desempeñando un papel crucial al innovar y traer nuevas ideas al campo de la IA. Además, instituciones académicas y organizaciones sin fines de lucro están contribuyendo con investigaciones de vanguardia que impulsan el desarrollo de la IA en diferentes direcciones.

## Noticias relevantes sobre Inteligencia Artificial

Los avances recientes en IA han tenido un impacto significativo en diversas industrias, desde la medicina hasta la logística. Los casos de uso exitosos de IA en el mundo real demuestran el potencial transformador de esta tecnología. Por otro lado, los eventos y conferencias relacionados con la IA ofrecen una oportunidad única para mantenerse al día con las últimas tendencias y conectarse con expertos en el campo.

En conclusión, la Inteligencia Artificial continúa siendo un área de gran interés y crecimiento en la era digital actual. Mantenerse informado sobre las últimas tendencias, los jugadores clave en el mercado y las noticias relevantes en el campo de la IA es crucial para aquellos interesados en aprovechar al máximo el potencial de esta tecnología. ¿Listo para sumergirte en el fascinante mundo de la IA? ¡Sigue nuestro blog para estar al tanto de todas las novedades!

### Palabras Clave de SEO: 
- Inteligencia Artificial
- Tendencias en IA
- Jugadores clave en IA
- Noticias de IA
- Aplicaciones de IA

Recursos:
- Informes de investigación de Gartner y Forrester.
- Artículos de TechCrunch y Wired.
- Publicaciones académicas y estudios de caso sobre aplicaciones exitosas de IA.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [27]:
topic = "Buenos Aires, Argentina"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Buenos Aires, Argentina.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: Ahora puedo dar una gran respuesta

Final Answer:

Plan de Contenido

Tema: Buenos Aires, Argentina

Introducción:
Buenos Aires, la capital de Argentina, es una ciudad vibrante y llena de historia, cultura y belleza. En este artículo, exploraremos los últimos acontecimientos, actores clave y noticias relevantes sobre esta fascinante ciudad.

Puntos Clave:
1. Últimas tendencias en Buenos Aires:
- Desarrollo urbano y arquitectónico
- Escena gastronómica y de entretenimiento
- Avances en tecnología y negocios

2. Actores clave en Buenos Aires:
- Go

I now can give a great answer

Final Answer: 

# Explorando Buenos Aires, Argentina

## Introducción

Buenos Aires, la capital de Argentina, es una ciudad vibrante y llena de historia, cultura y belleza. En este artículo, exploraremos los últimos acontecimientos, actores clave y noticias relevantes sobre esta fascinante ciudad.

## Últimas tendencias en Buenos Aires

Buenos Aires se encuentra en constante desarrollo urbano y arquitectónico, con nuevos proyectos que buscan combinar la modernidad con la rica historia de la ciudad. La escena gastronómica y de entretenimiento también está en auge, con una amplia variedad de restaurantes, bares y eventos culturales que satisfacen incluso a los paladares más exigentes. En cuanto a avances en tecnología y negocios, Buenos Aires se posiciona como un importante polo de innovación en América Latina, atrayendo a empresas y emprendedores de todo el mundo.

## Actores clave en Buenos Aires

El gobierno local y nacional juegan un papel fundamental e

In [28]:
Markdown(result)

# Explorando Buenos Aires, Argentina

## Introducción

Buenos Aires, la capital de Argentina, es una ciudad vibrante y llena de historia, cultura y belleza. En este artículo, exploraremos los últimos acontecimientos, actores clave y noticias relevantes sobre esta fascinante ciudad.

## Últimas tendencias en Buenos Aires

Buenos Aires se encuentra en constante desarrollo urbano y arquitectónico, con nuevos proyectos que buscan combinar la modernidad con la rica historia de la ciudad. La escena gastronómica y de entretenimiento también está en auge, con una amplia variedad de restaurantes, bares y eventos culturales que satisfacen incluso a los paladares más exigentes. En cuanto a avances en tecnología y negocios, Buenos Aires se posiciona como un importante polo de innovación en América Latina, atrayendo a empresas y emprendedores de todo el mundo.

## Actores clave en Buenos Aires

El gobierno local y nacional juegan un papel fundamental en el desarrollo y crecimiento de Buenos Aires, implementando políticas que buscan mejorar la calidad de vida de los ciudadanos. Asimismo, empresas líderes en diferentes sectores contribuyen al dinamismo económico de la ciudad, generando oportunidades de empleo y crecimiento. Las organizaciones culturales y artísticas también son parte fundamental de la identidad de Buenos Aires, promoviendo la diversidad y el talento local.

## Noticias destacadas en Buenos Aires

Los eventos culturales y deportivos son una parte esencial de la vida en Buenos Aires, con festivales, conciertos y competencias que atraen a visitantes de todas partes del mundo. Las iniciativas de sostenibilidad son cada vez más relevantes en la ciudad, con proyectos que buscan preservar el patrimonio natural y cultural de Buenos Aires. En cuanto a infraestructura y transporte, se han realizado importantes avances para mejorar la movilidad y conectividad de la ciudad, facilitando la vida diaria de sus habitantes.

## Conclusión

Buenos Aires es una ciudad dinámica y diversa que ofrece una amplia gama de experiencias para los visitantes y residentes por igual. Al mantenernos al tanto de las últimas tendencias, actores clave y noticias destacadas, podemos apreciar plenamente todo lo que esta ciudad tiene para ofrecer.

¡Planee su próximo viaje a Buenos Aires y explore todo lo que esta increíble ciudad tiene para ofrecer! ¡No se pierda las últimas noticias y eventos emocionantes que están sucediendo en la capital argentina!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).